In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import Audio
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.playback import play
from tqdm.notebook import tqdm

In [9]:
# Imports
from scipy.io import wavfile

# Your new sampling rate
new_rate = 22050

sound = AudioSegment.from_file(
    "data/train_wavs/dia0_utt0.wav").set_frame_rate(new_rate)


In [21]:
def resample_audio(indir, outdir, new_rate=22050):
    """Changes sample rate of audio with minimal quality loss."""
    !mkdir data/train_wavs_22050
    for file in tqdm(os.listdir(indir)):
        if not file.startswith('.'):
            sound = AudioSegment.from_file(
                f"{indir}/{file}").set_frame_rate(new_rate).export(f"{outdir}/{file}", format="wav")

resample_audio("data/train_wavs", "data/train_wavs_22050")

In [4]:
CHARACTERS = ['Joey']
df = pd.read_csv("data/train_sent_emo.csv")

In [5]:
df = df.loc[df.Speaker == 'Joey']

In [6]:
df['filename'] = [f"train_wavs_22050/dia{i[0]}_utt{i[1]}.wav" for i in np.column_stack((df.Dialogue_ID.values, df.Utterance_ID.values))]

In [7]:
df = df.filter(['Utterance', 'filename'])

In [8]:
dataset = list(map(lambda text, file: f"{file}|{text}", df.Utterance.values, df.filename.values))

In [9]:
from sklearn.model_selection import train_test_split

train, test_val = train_test_split(dataset, test_size=0.046)
test, val = train_test_split(test_val, test_size=0.2)
len(train), len(test), len(val)

(1439, 56, 14)

In [10]:
import re
def restore_windows_1252_characters(s):
    """Replace C1 control characters in the Unicode string s by the
    characters at the corresponding code points in Windows-1252,
    where possible.

    """
    import re
    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, s)

In [11]:
train = restore_windows_1252_characters("\n".join(train))

In [12]:
# write training set to file
with open("joeyspeech_train_filelist.txt", "w+") as train_file:
    train_file.write(train)

In [13]:
# write validation set to file
val = restore_windows_1252_characters("\n".join(val))
with open("joeyspeech_val_filelist.txt", "w+") as val_file:
    val_file.write(val)

In [14]:
# write test set to file
test = restore_windows_1252_characters("\n".join(test))
with open("joeyspeech_test_filelist.txt", "w+") as test_file:
    test_file.write(test)

In [19]:
wavfile.read("data/train_wavs_22050/dia0_utt0.wav")[1][:,1].shape

(125127,)

In [17]:
wavfile.read("data/train_wavs_22050/LJ001-0001.wav")[1].shape

(212893,)

In [18]:
125127*2

250254

In [22]:
# convert all audio to single-channel
for file in tqdm(os.listdir("data/train_wavs_22050")):
    
    arr = wavfile.read(f"data/train_wavs_22050/{file}")[1][:,1]
    wavfile.write(f"data/train_wavs_22050/{file}", 22050, arr)